# Initialize_agent Vs AgentExecutor

| Think of it like:                                      |
| ------------------------------------------------------ |
| `initialize_agent()` → ordering a preset meal combo    |
| `AgentExecutor` → building a meal from raw ingredients |


# run() vs invoke()

| Feature          | `.run()`                          | `.invoke()`                           |
| ---------------- | --------------------------------- | ------------------------------------- |
| Input format     | Single string or dict             | Dict only                             |
| Output format    | Single string                     | Dict (can have multiple keys)         |
| Multiple outputs | ❌ Not supported                   | ✅ Fully supported                     |
| Error handling   | ❌ Prone to `ValueError` if >1 key | ✅ Robust with flexible outputs        |
| Use case         | Simple chain calls                | Agents, tools, tracing, custom chains |
| Modern interface | 🚫 Legacy method                  | ✅ New `Runnable` standard             |


If you're using LangChain ≥ v0.1, always prefer .invoke() for flexibility and future-proofing.

# the chain which returns citations

| **Chain / Retriever**               | **Returns Citations** | **Method / Output Key**                          | **Status**             | **Docs Link**                                                                               |
| ----------------------------------- | --------------------- | ------------------------------------------------ | ---------------------- | ------------------------------------------------------------------------------------------- |
| `RetrievalQAWithSourcesChain`       | ✅ Yes                 | `result['answer']`, `result['sources']`          | **Stable**             | [🔗 Docs](https://docs.langchain.com/docs/modules/chains/popular/retrieval-qa-with-sources) |
| `ConversationalRetrievalChain`      | ✅ Yes                 | `result['answer']`, `result['source_documents']` | **Stable**             | [🔗 Docs](https://docs.langchain.com/docs/modules/chains/popular/chat_vector_db)            |
| `MultiRetrievalQAChain`             | ✅ Yes                 | Depends on sub-chains used                       | **Stable**             | [🔗 Docs](https://docs.langchain.com/docs/modules/chains/popular/multi_retrieval_qa)        |
| `VectorDBQAWithSourcesChain`        | ✅ Yes                 | `result['answer']`, `result['sources']`          | ✅ (but legacy)         | [🔗 Docs](https://docs.langchain.com/docs/modules/chains/popular/vector-db-qa)              |
| `Tool` using RetrievalQAWithSources | ✅ Yes                 | `result['answer']`, `result['sources']`          | **Stable** (via Agent) | [🔗 Tools Docs](https://docs.langchain.com/docs/modules/agents/tools/custom_tools)          |
| `RetrievalQA` (basic chain)         | ❌ No                  | Only `answer`                                    | **Stable**             | [🔗 Docs](https://docs.langchain.com/docs/modules/chains/popular/retrieval-qa)              |
| `RefineDocumentsChain`              | ❌ No                  | Only `answer`                                    | **Stable**             | [🔗 Docs](https://docs.langchain.com/docs/modules/chains/document/refine)                   |
| `StuffDocumentsChain`               | ❌ No                  | Only `answer`                                    | **Stable**             | [🔗 Docs](https://docs.langchain.com/docs/modules/chains/document/stuff)                    |


In [2]:
# https://python.langchain.com/docs/how_to/#retrievers

* Purpose
* Input / Output
* Unique behavior
* Ideal use cases
* Key differences
* Example usage context

---

### ✅ **LangChain Retriever Comparison Table**

| **Retriever**                        | **Purpose**                                                       | **Input**              | **Output**                                | **Key Behavior**                                                           | **Ideal Use Case**                                 | **What Makes It Unique**                        | **Example Scenario**                               |
| ------------------------------------ | ----------------------------------------------------------------- | ---------------------- | ----------------------------------------- | -------------------------------------------------------------------------- | -------------------------------------------------- | ----------------------------------------------- | -------------------------------------------------- |
| **ContextualCompressionRetriever**   | Compresses retrieved docs for brevity using LLM                   | Query string           | Condensed relevant docs                   | Applies LLM to shorten retrieved docs before passing to QA                 | Long documents with lots of irrelevant data        | Reduces token usage and improves response time  | Customer support knowledge base                    |
| **ParentDocumentRetriever**          | Retrieve full parent doc containing relevant chunk                | Query string           | Full parent documents                     | Stores smaller chunks in vectorstore, retrieves parent doc from ID mapping | When context is split but full doc needed          | Keeps hierarchical relation of chunk→parent     | Legal contracts or PDFs with sections              |
| **EnsembleRetriever**                | Combines results from multiple retrievers                         | Query string           | Ranked set of docs                        | Aggregates and scores results (e.g., hybrid BM25 + vector)                 | When combining traditional + semantic search       | Balances recall and relevance                   | Search with both keyword match and meaning         |
| **BM25Retriever**                    | Traditional keyword-based retrieval using BM25 algo               | Query string           | Ranked documents                          | Uses term frequency + inverse document frequency scoring                   | Classic keyword search                             | No embeddings required, fast                    | Legal or scientific keyword-heavy search           |
| **MultiQueryRetriever**              | Expands query into multiple subqueries via LLM                    | Query string           | Combined relevant docs                    | Generates multiple perspectives for a query                                | Broad/ambiguous user queries                       | Boosts recall across varied contexts            | "What is LangChain?" → benefits, history, features |
| **SelfQueryRetriever**               | Structured querying over metadata + vector store                  | Natural language query | Filtered docs based on metadata + content | Parses query into metadata filters + semantic query                        | When metadata (e.g., date, type) is important      | Uses LLM to understand structured intent        | "Find blogs after 2023 about GenAI"                |
| **TimeWeightedVectorStoreRetriever** | Returns recent & relevant documents                               | Query string           | Docs sorted by recency & relevance        | Applies decay over time to prioritize fresh data                           | Chatbot with memory or streaming context           | Learns from past interactions with weight decay | Conversational memory bot                          |
| **KNNRetriever**                     | Basic similarity search using k-nearest neighbors                 | Query vector           | Top-k similar documents                   | Pure vector search with similarity scoring                                 | Simple vector search tasks                         | Very fast & simple                              | Searching similar documents by vector distance     |
| **TFIDFRetriever**                   | Traditional TF-IDF based keyword retrieval                        | Query string           | Ranked documents                          | Ranks based on tf-idf scores                                               | Lightweight traditional search                     | Simpler than BM25                               | Quick text file search                             |
| **SVMRetriever**                     | Uses Support Vector Machine model for classification-based search | Vectorized input       | Matched documents                         | ML-based approach to retrieval                                             | When classification is needed over embedding space | Requires training, not plug-and-play            | Spam detection or classifying docs by intent       |

---

### 🧠 Key Differences & Decision Points

| **Question**                                           | **Best Retriever(s)**                      | **Why**                                 |
| ------------------------------------------------------ | ------------------------------------------ | --------------------------------------- |
| Need to retrieve full document but store small chunks? | `ParentDocumentRetriever`                  | Maintains full document structure       |
| Need to boost keyword + vector search?                 | `EnsembleRetriever`, `MultiQueryRetriever` | Hybrid + multi-angle querying           |
| Need fast keyword search with no embeddings?           | `BM25Retriever`, `TFIDFRetriever`          | Classical search, lightweight           |
| Need structured metadata-based filtering?              | `SelfQueryRetriever`                       | LLM parses query for structured filters |
| Want more recent documents prioritized?                | `TimeWeightedVectorStoreRetriever`         | Adds recency decay factor               |
| Need LLM to compress and summarize documents?          | `ContextualCompressionRetriever`           | Saves token + removes noise             |

---



| Chain Name                       | Description                                                       | Input                   | Output                      | Use Case Example                       |
| -------------------------------- | ----------------------------------------------------------------- | ----------------------- | --------------------------- | -------------------------------------- |
| **LLMChain**                     | Basic chain using prompt + LLM.                                   | Text (prompt variables) | Text (response)             | Prompt templating, response generation |
| **SimpleSequentialChain**        | Chains executed one after another (output of 1 is input to next). | Text                    | Final output                | Step-by-step generation                |
| **SequentialChain**              | Like above but supports multiple inputs/outputs.                  | Dict (inputs)           | Dict (outputs)              | Complex multi-step pipelines           |
| **RetrievalQA**                  | Adds retriever to LLMChain for document-based QA.                 | Query string            | Answer string               | RAG-style simple QA                    |
| **RetrievalQAWithSourcesChain**  | Same as above but returns sources.                                | Query string            | Dict with answer + sources  | QA with citation-based responses       |
| **MapReduceDocumentsChain**      | Applies map-reduce logic to documents.                            | List of Documents       | Summary string              | Long doc summarization                 |
| **StuffDocumentsChain**          | Stuffs all docs into a single prompt.                             | List of Documents       | Answer/summary              | Small doc summarization or synthesis   |
| **RefineDocumentsChain**         | Iteratively refines answers based on docs.                        | List of Documents       | Answer string               | Stepwise improvement of response       |
| **ConversationalRetrievalChain** | RetrievalQA with chat history/memory                              | Question + history      | Answer + sources (optional) | Conversational chatbot with doc memory |
| **MultiPromptChain**             | Routes queries to different prompts based on input intent.        | Text                    | Text                        | Dynamic prompt routing                 |
| **LLMSummarizationCheckerChain** | Validates whether an LLM summary is faithful to the input docs.   | Summary + documents     | True/False or score         | Quality control of summarization       |
| **APIChain**                     | Makes external API calls using LLM-generated requests.            | Query string            | Parsed API response         | Weather, News, etc. from external APIs |
| **SQLDatabaseChain**             | Translates natural language to SQL and queries databases.         | Natural question        | Query result string         | Ask questions to structured databases  |


| **Agent Types (Exact Name)**                                       | **Chain Types**                | **Retriever Types**                | **Memory Types**                  |
| ------------------------------------------------------------------ | ------------------------------ | ---------------------------------- | --------------------------------- |
| `AgentType.ZERO_SHOT_REACT_DESCRIPTION`                            | `LLMChain`                     | `VectorStoreRetriever`             | `ConversationBufferMemory`        |
| `AgentType.CONVERSATIONAL_REACT_DESCRIPTION`                       | `RetrievalQA`                  | `MultiQueryRetriever`              | `ConversationSummaryMemory`       |
| `AgentType.OPENAI_FUNCTIONS`                                       | `ConversationalRetrievalChain` | `ParentDocumentRetriever`          | `ConversationBufferWindowMemory`  |
| `AgentType.OPENAI_TOOLS`                                           | `MapReduceDocumentsChain`      | `EnsembleRetriever`                | `ConversationSummaryBufferMemory` |
| `AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION`            | `StuffDocumentsChain`          | `BM25Retriever`                    | `ZepMemory`                       |
| `AgentType.OPENAI_MULTI_FUNCTIONS` (LangGraph or Multi-call Tools) | `RefineDocumentsChain`         | `SelfQueryRetriever`               | `PostgresChatMessageHistory`      |
| `AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION`                  | `RetrievalQAWithSourcesChain`  | `TimeWeightedVectorStoreRetriever` | `VectorStoreRetrieverMemory`      |
| `AgentExecutor` (Custom from `initialize_agent`)                   | `VectorDBQA`                   | `ContextualCompressionRetriever`   | `MotorheadMemory`                 |
|                                                                    | `MapReduceChain`               | `TavilySearchAPIRetriever`         | `FileChatMessageHistory`          |
|                                                                    | `AnalyzeDocumentChain`         | `WebResearchRetriever`             |                                   |
|                                                                    | `QAWithSourcesChain`           |                                    |                                   |
